https://www.recurse.com/pairing-tasks

> Lisp parser

> Write code that takes some Lisp code and returns an abstract syntax tree. The AST should represent the structure of the code and the meaning of each token. For example, if your code is given "(first (list 1 (+ 2 3) 9))", it could return a nested array like ["first", ["list", 1, ["+", 2, 3], 9]].

> During your interview, you will pair on writing an interpreter to run the AST. You can start by implementing a single built-in function (for example, +) and add more if you have time.

In [1]:
# first pass: deal with only spaces, ( and ) as reserved characters for tokenizing a string

TEST_INPUT = "(first (list 1 (+ 2 3) 9))"
TEST_OUTPUT = ["first", ["list", 1, ["+", 2, 3], 9]]

In [2]:
def raw_tokenize(s):
    # replace ( and ) with surrounding spaces 
    s = s.replace('(', ' ( ').replace(')', ' ) ')
    return s.split()

# convert a string to a number if possible
# in Python, what are the native numeric types?

def to_int(s):
    try:
        return int(s)
    except:
        return s

def typify_token(s):
    """
    for now, convert the token to an int if possible, otherwise leave token as a string
    """
    return to_int(s)

def tokenize(s):
    return [typify_token(t) for t in raw_tokenize(s)]


In [3]:
tokens = tokenize("first (list 1 (+ 2 3) 9))")
tokens

['first', '(', 'list', 1, '(', '+', 2, 3, ')', 9, ')', ')']

In [7]:
def ast_from_tokens(tokens):

    stack = []

    for (i, token) in enumerate(tokens):
        if token == '(':
            stack.append([])
        elif token == ')':
            # pop the list on the top of the stack
            # and append it to the end of the preceding layer
            try:
                top = stack.pop()
            except Exception as e:
                raise Exception("Closing ) without corresponding (")
            else:    
                if len(stack) > 0:
                    stack[-1].append(top)
                else:
                    if i+1 == len(tokens):
                        return top
                    else:
                        raise Exception("Extra tokens beyond valid expression")
                
        else:
            # append item to the list at the top of the stack
            try:
                stack[-1].append(token)
            except Exception as e:
                raise Exception("Missing opening (")

    if len(stack) > 0:
        raise Exception("Missing closing )")


In [8]:
ast_from_tokens(tokenize(TEST_INPUT))

['first', ['list', 1, ['+', 2, 3], 9]]

In [9]:
ast_from_tokens(tokenize("(first list 1 (+ 2 3) 9"))

In [ ]:
# using Claude 2 -- response to can I make use of Python's ast library to distinguish strings that look like int vs float?
# then I asked for a fix

import ast

def int_or_float(s):
  try:
    node = ast.parse(s)
  except SyntaxError:
    return 'unknown'

  if isinstance(node.body[0].value, ast.Num):
    if isinstance(node.body[0].value.n, int):
      return 'int'
    elif isinstance(node.body[0].value.n, float):
      return 'float'
    else:
      return 'unknown'
  else:
    return 'unknown'

print(int_or_float('42')) # int
print(int_or_float('3.14')) # float  
print(int_or_float('abc')) # unknown